In [39]:
import pandas as pd
import os
from statsmodels.tsa.arima.model import ARIMA
import pickle
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu chi tiêu và phân cụm
spending_data = pd.read_csv('../user_data_ver1/jars_distribution_with_actual.csv')
cluster_data = pd.read_csv('../user_classification/user_clusters.csv')

# Gộp dữ liệu chi tiêu với nhãn cụm
merged_data = pd.merge(spending_data, cluster_data, on='user_id', how='left')

# Áp dụng trọng số cho 12 tháng (ưu tiên 3 tháng gần nhất: 10, 11, 12)
merged_data['weight'] = merged_data['month'].apply(lambda x: 1.0 if x in [10, 11, 12] else 0.5)

# Chuẩn bị chuỗi thời gian có trọng số cho từng hũ hoặc cụm
def prepare_weighted_time_series(data_df):
    weighted_series = data_df.groupby('month').apply(
        lambda x: (x['actual_spent_amount'] * x['weight']).sum() / x['weight'].sum()
    ).sort_index()
    return weighted_series

# Huấn luyện ARIMA cho từng hũ trong từng cụm và lưu mô hình
core_jars = ["NEC", "FFA", "EDU", "LTSS", "PLAY"]
models = {}

for cluster_id in range(4):
    cluster_df = merged_data[merged_data['cluster'] == cluster_id]
    if cluster_df.empty:
        print(f"Không có dữ liệu cho cụm {cluster_id}")
        continue
    
    # Tạo thư mục nếu chưa tồn tại
    os.makedirs("arima_models", exist_ok=True)

    # Huấn luyện cho toàn cụm
    # ts_cluster = prepare_weighted_time_series(cluster_df)
    # if len(ts_cluster) < 2:
    #     print(f"Không đủ dữ liệu cho cụm {cluster_id}")
    #     continue
    
    # try:
    #     model = ARIMA(ts_cluster, order=(1, 0, 1))
    #     model_fit = model.fit()
    #     models[cluster_id] = model_fit
    #     # Lưu mô hình cho cụm
    #     with open(f"arima_models/cluster_{cluster_id}_arima.pkl", "wb") as f:
    #         pickle.dump(model_fit, f)
    #     print(f"Mô hình ARIMA cho cụm {cluster_id} đã được huấn luyện")
    # except Exception as e:
    #     print(f"ARIMA không hội tụ cho cụm {cluster_id}: {e}")

    # Huấn luyện cho từng hũ trong cụm
    for jar in core_jars:
        jar_data = cluster_df[cluster_df['jar'] == jar]
        if len(jar_data) == 0:
            print(f"Không có dữ liệu cho hũ {jar} trong cụm {cluster_id}")
            continue
        
        ts_jar = prepare_weighted_time_series(jar_data)
        if len(ts_jar) < 2:
            print(f"Không đủ dữ liệu cho hũ {jar} trong cụm {cluster_id}")
            continue
        
        try:
            model = ARIMA(ts_jar, order=(1, 0, 1))
            model_fit = model.fit()
            models[(cluster_id, jar)] = model_fit
            # Lưu mô hình cho từng hũ
            with open(f"arima_models/cluster_{cluster_id}_{jar.lower()}_arima.pkl", "wb") as f:
                pickle.dump(model_fit, f)
            print(f"Mô hình ARIMA cho hũ {jar} trong cụm {cluster_id} đã được huấn luyện")
        except Exception as e:
            print(f"ARIMA không hội tụ cho hũ {jar} trong cụm {cluster_id}: {e}")

# Hàm dự đoán chi tiêu cho tháng tiếp theo


Mô hình ARIMA cho hũ NEC trong cụm 0 đã được huấn luyện
Mô hình ARIMA cho hũ FFA trong cụm 0 đã được huấn luyện
Mô hình ARIMA cho hũ EDU trong cụm 0 đã được huấn luyện
Mô hình ARIMA cho hũ LTSS trong cụm 0 đã được huấn luyện
Mô hình ARIMA cho hũ PLAY trong cụm 0 đã được huấn luyện
Mô hình ARIMA cho hũ NEC trong cụm 1 đã được huấn luyện
Mô hình ARIMA cho hũ FFA trong cụm 1 đã được huấn luyện
Mô hình ARIMA cho hũ EDU trong cụm 1 đã được huấn luyện
Mô hình ARIMA cho hũ LTSS trong cụm 1 đã được huấn luyện
Mô hình ARIMA cho hũ PLAY trong cụm 1 đã được huấn luyện
Mô hình ARIMA cho hũ NEC trong cụm 2 đã được huấn luyện
Mô hình ARIMA cho hũ FFA trong cụm 2 đã được huấn luyện
Mô hình ARIMA cho hũ EDU trong cụm 2 đã được huấn luyện
Mô hình ARIMA cho hũ LTSS trong cụm 2 đã được huấn luyện
Mô hình ARIMA cho hũ PLAY trong cụm 2 đã được huấn luyện
Mô hình ARIMA cho hũ NEC trong cụm 3 đã được huấn luyện
Mô hình ARIMA cho hũ FFA trong cụm 3 đã được huấn luyện
Mô hình ARIMA cho hũ EDU trong cụm 3 đã đư

In [44]:
def predict_next_month_spending(cluster_id, jar=None, month=13):
    key = (cluster_id, jar) if jar else cluster_id
    if key not in models:
        print(f"Không có mô hình cho cụm {cluster_id}" + (f" hoặc hũ {jar}" if jar else ""))
        return None
    
    model = models[key]
    forecast = model.forecast(steps=1).iloc[0]  # Sử dụng iloc[0] để lấy giá trị
    # Làm tròn đến 100,000
    rounded_forecast = round(forecast / 100000) * 100000
    return rounded_forecast

# Dự đoán cho từng hũ trong cụm 0
for i in range(0,4):
    sum = 0
    print(f"\nDự đoán chi tiêu cho cụm {i} (làm tròn đến 100k):")
    for jar in core_jars:
        prediction = predict_next_month_spending(i, jar)
        if prediction is not None:
            sum = sum + prediction
            print(f"Hũ {jar}: {prediction:.2f} VND")

    # Dự đoán tổng quát cho cụm 0
    prediction_cluster = sum
    if prediction_cluster is not None:
        print(f"Tổng chi tiêu trung bình cụm 0: {prediction_cluster:.2f} VND")


Dự đoán chi tiêu cho cụm 0 (làm tròn đến 100k):
Hũ NEC: 7500000.00 VND
Hũ FFA: 2100000.00 VND
Hũ EDU: 1500000.00 VND
Hũ LTSS: 2100000.00 VND
Hũ PLAY: 1800000.00 VND
Tổng chi tiêu trung bình cụm 0: 15000000.00 VND

Dự đoán chi tiêu cho cụm 1 (làm tròn đến 100k):
Hũ NEC: 8200000.00 VND
Hũ FFA: 2400000.00 VND
Hũ EDU: 1800000.00 VND
Hũ LTSS: 1800000.00 VND
Hũ PLAY: 1800000.00 VND
Tổng chi tiêu trung bình cụm 0: 16000000.00 VND

Dự đoán chi tiêu cho cụm 2 (làm tròn đến 100k):
Hũ NEC: 28100000.00 VND
Hũ FFA: 7100000.00 VND
Hũ EDU: 5200000.00 VND
Hũ LTSS: 5300000.00 VND
Hũ PLAY: 5500000.00 VND
Tổng chi tiêu trung bình cụm 0: 51200000.00 VND

Dự đoán chi tiêu cho cụm 3 (làm tròn đến 100k):
Hũ NEC: 7500000.00 VND
Hũ FFA: 1500000.00 VND
Hũ EDU: 1100000.00 VND
Hũ LTSS: 1300000.00 VND
Hũ PLAY: 1200000.00 VND
Tổng chi tiêu trung bình cụm 0: 12600000.00 VND
